### __1) Write a python function which take a dataframe as input and deals with the issue of outliers in all the continuous variables.__


__Solution :__ The function _OutlrTrtmnt(cont)_ will find the continuous values and then, remove the outliers through Inter Quantile Range technique. Although, it is a very basic technique but, robust to work as standalone pack with different kinds of datasets.

Impovisations: Adding 2-3 more outlier techniques would make the function a good auto-outlier detection(likely) library\pack.

In [ ]:
###Importing Packages
import pandas as pd
import numpy as np
from google.colab import drive
#drive.mount('/content/gdrive/')
from sklearn.preprocessing import StandardScaler
"""Assumptions for the dataset:
   1. Data is partially preprocessed(apart from outliers treatment)
   2. There are no missing or NaN values""" 

  
### Loading Data
Data = pd.read_csv(input("Please input Dataset link"))         # Runtime input is to be provided
df = pd.DataFrame(Data)
#df.head()                                                      # For vieweing the input DataFrame

df = df.replace(np.NaN, df.mean())                             # A basic preprocessing step of removing NaN values.

### Finding continuous data columns in the Dataset
iscont = {}                                                    # Dictionary will store the names of coninuous columns
for col in df.columns:
    iscont[col] = df[col].nunique() > (0.05/df[col].count())   # If uniques in the data are more than 5% of rows then, the column is considered as continuous
#The boolean output will be stored inside 'iscont' dict to compare and identify continuuos columns. Where, True -> more than 5% uniques and False -> less than 5%uniques.
#To cover all kinds of data, 5% has been choosen as threshold.

cont = pd.DataFrame()                                          # Initializing continuous columns dataframe.
count = 0                                                      # initializing count for message, if there are continuous data columns in the dataset
for col in df.columns:    
  if ((iscont[col]==True) & (df[col].dtype == float)):         # If uniques are greater than 5%(which is True) and Column type is float then, the column in considered as contiuous 
    cont[col] = df[col]                                        # 'cont' is the dataframe with all the continuous data columns. 
    count +=1
if(count<1):
    print('There are no continuous data columns in the given dataset')

#cont.head()                                                    # For viewing the seperated Continuous DataFrame

### Defining a function for removing outliers using Quantile Ranges:
def OutlrTrtmnt(cont):
    Q1 = cont.quantile(0.10)                                   # Calculating Q1 value using quantile(0.10)
    Q3 = cont.quantile(0.90)                                   # Calculating Q3 value using quantile value (0.90)
    Q = Q3-Q1                                                  # Calculating Inter Quartile Range
    NoOutlr = cont[~((cont<(Q1-Q))|(cont>(Q3+Q))).any(axis=1)] # Checking for outliers and droping them off
    return NoOutlr                                             # returning the created variable

### Scaling continuous column dataset so that, all the columns are in same scale while removing outliers.
scaler = StandardScaler().fit(cont) 
scaledData = scaler.transform(cont)

cont = OutlrTrtmnt(cont)
print(len(df)-len(cont), "Outliers were found and removed through Quantile Range")


### __2) Write a function in python that inputs a dataframe and identify which columns have date in them. Using these date columns make new columns which are difference between these columns taking 2 at a time. (for instance if there is date1, date2, date3 columns, output should be like date1-date2, date2-date3, date1-date3 )__

__Solution :__ The random date dataset is created and input to the function _diffofdates(a,b)_ for identifying the date columns. Once found, all the dates columns undergo differences inbetween with two at a time. 

The function is built efficient to any kind of date dataset. And the output of the function is a dataframe with columns names as the date columns combinations.

In [ ]:
#Importing Packages
import pandas as pd
import numpy as np
from google.colab import drive
#drive.mount('/content/gdrive/')
from datetime import date
from itertools import combinations

### Generating random date dataframe as an input
np.random.seed(0)
rnge1 = pd.date_range('1980-01-01','2019-12-31', periods=50)
rnge2 = pd.date_range('1956-01-01', '2009-12-31',periods=50)
rnge3 = pd.date_range('1856-01-01', '2020-07-24',periods=50)
rnge4 = pd.date_range('2000-01-01', '2009-12-31',periods=50)
rnge5 = pd.date_range('1992-04-17', '1994-07-15',periods=50)
df = pd.DataFrame({ 'Date1': rnge1,'Date2': rnge2,'Date3': rnge3,'Date4': rnge4,'Date5': rnge5,'Value1': np.random.randn(len(rng1)), 'Value2': np.random.randn(len(rng2))}) 


#df.head()                                                      # For vieweing the input DataFrame
#Note: Here, the numeric columns: Value1 and Value2 are created only to show that the function can seperate the date from any kind of given dataset.


### Finding and converting 'date' columns to data type -> 'datetime64[ns]'
for col in df.columns:
  if df[col].dtype == 'object':                                # First step of filtering using default 'object' data type 
    try:
      df[col] = pd.to_datetime(df[col])                        # With try-except, the data type of date columns have been changed to 'datetime64[ns]'
    except ValueError: 
      pass



### Finding and seperating date data columns from the dataset
od=pd.DataFrame()                                              # Initializing dataframe for date columns
for col in df.columns:
  if df[col].dtype == '<M8[ns]':                               # if the dtype of column is '<M8[ns]'(which datetime dtype) then, the column is considered to be date column
    od[col] = [d.date() for d in df[col]]                      
    od[col] = df[col].dt.strftime('%Y/%m/%d')                  # Tuning various date formats in the dataset into single format -> '%Y/%m/%d'
    od[col] = pd.to_datetime(od[col])                          # In the date columns only dataframe, the dtype is defaulted to object. So, we change it back to 'datetime64[ns]'. because, only then, we can perform date arthimetic operations.


#od.head()                                                      # Display of identified date columns

lst = list(combinations(od.columns,2))                         # Created a list for binding all the columns 2 at a time.
#lst                                                           # Display of combinations in the list
""" The list helps in creating the arthimetic combinations among the columns. Ex: if a,b,c are the columns.
    lst gives the combination such as, (a,b), (b,c), (c,a) and these are passed to function"""
lst

### Defining a function for performing arthematic operations on the date columns to find days
dd=pd.DataFrame()
def diffofdates(a,b):                                          # Functions with two parameters: a-> first date column and b -> second date column for first iteration as seen in above example.
  for i in lst:
    dd[str(i)]= (od[a] - od[b]).dt.days                        # One-by-one all the combinations will undergo difference operation

### Run the function
for a,b in lst:                                                # Looping over the combination to pass values to the function
  diffofdates(a,b)                                             # Finally triggering the function


### The output finds the date columns and gives the difference of the dates in dataset
dd.head()

## __3) Write a function in python that take dataframe as input and drop columns having Pearson correlation more than 0.85__


__Solution:__ Once given the dataset, the function evaluates correlation among the columns. Considering the threshold for 0.85, all the columns above it are dropped. The output of the function is dataframe with columns having pearson correlation less than 0.85.

In [ ]:
#Importing Packages
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive/')
"""Assumptions for the dataset:
   1. Data is partially preprocessed(apart from outliers treatment)
   2. There are no missing or NaN values"""

### Loading Data
Data = pd.read_csv(input("Please input Dataset link"))                                # Runtime input is to be provided
df = pd.DataFrame(Data)

df = df.replace(np.NaN, df.mean())                                                    # A basic preprocessing step of removing NaN values.

#df.head()
 
corr_matrix = df.corr().abs()                                                         # Creating correlation matrix
#corr_matrix                             
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))   # Filtering the upper triangle of correlation matrix using numpys

to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]           # Finding index of feature column which have pearson correlation > 0.85
''' Every column is checked if its correlation is more than 85 or not. If yes, then it will dropped'''
df.drop(df[to_drop], axis=1)                                                          # Dropping the features and displaying the modified dataset 